# Try 16: setup `paprika` restraints (confirm-original)

Presuming we have a host-guest system already setup, let's see if we can rationally instruct `paprika` to setup all the restraints for a real system.

In [1]:
%load_ext autoreload
%autoreload 2

import os as os
import numpy as np

import parmed as pmd
import pytraj as pt

In [2]:
import logging
import datetime as dt
d_date = dt.datetime.now()
logging.basicConfig(filename='paprika' + '-' + d_date.strftime("%Y-%m-%d-%I-%M-%S-%p") + '.log', 
                    format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.DEBUG)
logging.info('Started logging...')

import paprika
print(paprika.__version__)

from paprika.restraints import static_DAT_restraint
from paprika.restraints import DAT_restraint
from paprika.restraints import amber_restraint_line
from paprika.restraints import create_window_list

from paprika.utils import make_window_dirs

2018-04-02_15:06:19_-0700-5a6c2cb-0.0.3


## Preamble

We are going to setup host translational restraints, guest translational restraints, host "jacks," and guest wall restraints with `paprika`. First, let's try working in a directory where we have a known `disang.rest` file we can use as a reference: `/data/davids4/projects/smirnoff-host-guest/a-bam-p/a00/original` or `/data/davids4/projects/smirnoff-host-guest/a-bam-p/u00/original` for the final values.

Since we're starting with files Niel prepared, the dummy atoms are first, followed by the host, and then the guest atoms.

<img src="images/paprika.png?2">

In [3]:
dummy_anchors = [':1', ':2', ':3']
host_anchors  = [':4@O3', ':6@C1', ':8@C6']
guest_anchors = [':10@C4', ':10@N1']

In [6]:
hg = pmd.load_file('systems/a-bam-p/original/a00/original/full.hmr.topo',
                   'systems/a-bam-p/original/a00/original/full.crds',
                    structure=True)

The attach fractions can be found in the `Setup.pl` file in the original location on `kirkwood`: `/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00/` under `@AttachFC`.

The attach force constant is 5.0 kcal/mol-A^{2} for distances and 100.0 kcal/mol-rad^{2} for angels and torsions based on `disang.rest` in `u00`.

The pull distances are specified by `@Translate` in `Setup.pl` and are offset by 6 Angstroms.

In [7]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 16.80 17.20 17.60 18.00'
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

windows = [len(attach_fractions), len(pull_distances), 0]
print(f'There are {windows} windows in this attach-pull calculation.')

There are [15, 46, 0] windows in this attach-pull calculation.


## Define the host translational restraints
These restraints help pin down the host during initial attachment of the DAT restraints. They are static, meaning unlike other restraints, they are not turned on slowly during the attachment phase.

In [8]:
static_restraint_atoms = [[dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[2], dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1], host_anchors[2]],
                        ]

static_restraint_distance_fc = 5.0
static_restraint_angle_fc = 100.0

## Define the guest translational restraints

In [9]:
guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

guest_restraint_targets = [6.0,
                           180.0,
                           180.0
                          ]
guest_restraint_target_final = [24.0, 180.0, 180.0]
guest_restraint_distance_fc = 5.0
guest_restraint_angle_fc = 100.0

## Define the host conformational restraints

The "jacks" restraints can be written O5_n - C1_n - O1_n - C4_n+1 and C1_n - O1_n - C4_n+1 - C5_n+1.

In [10]:
host_conformational_template = [['O5', 'C1', 'O1', 'C4'],
                                ['C1', 'O1', 'C4', 'C5']
                               ]

host_residues = len(hg[':MGO'].residues)
first_host_residue = hg[':MGO'].residues[0].number + 1
conformational_restraint_atoms = []
conformational_restraint_targets = []
conformational_restraint_fc = 6.0

for n in range(first_host_residue, host_residues + first_host_residue):
    if n + 1 < host_residues + first_host_residue:
        next_residue = n + 1
    else:
        next_residue = first_host_residue
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[0][0]}',
         f':{n}@{host_conformational_template[0][1]}',
         f':{n}@{host_conformational_template[0][2]}',
         f':{next_residue}@{host_conformational_template[0][3]}'
        ])
    conformational_restraint_targets.append(104.30)
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[1][0]}',
         f':{n}@{host_conformational_template[1][1]}',
         f':{next_residue}@{host_conformational_template[1][2]}',
         f':{next_residue}@{host_conformational_template[1][3]}'
        ])
    conformational_restraint_targets.append(-108.8)

## Define the guest "wall" restraints

The guest wall restraints are only on during the attachment phase, and make sure that the guest does not unbind and waste time diffusing around the simulation box. These distance restraints are flat from zero through the restraint target value, near the edge of the binding cavity, and then increase harmonically beyond that. The guest "wall" angle restraint is harmonic and prevents the guest from flipping around during the bound phase.

In [11]:
guest_wall_template = [
    ['O2', guest_anchors[0]],
    ['O6', guest_anchors[0]]
]

guest_wall_restraint_atoms = []
guest_wall_restraint_targets = []
guest_wall_restraint_angle_fc = 500.0
guest_wall_restraint_distance_fc = 50.0

for n in range(first_host_residue, host_residues + first_host_residue):
    guest_wall_restraint_atoms.append(
        [f':{n}@{guest_wall_template[0][0]}',
         f'{guest_wall_template[0][1]}',
        ])
    guest_wall_restraint_targets.append(11.3)
    guest_wall_restraint_atoms.append(
        [f':{n}@{guest_wall_template[1][0]}',
         f'{guest_wall_template[1][1]}',
        ])
    guest_wall_restraint_targets.append(13.3)

guest_wall_restraint_atoms.append([dummy_anchors[1],
                                 guest_anchors[0],
                                 guest_anchors[1]])
guest_wall_restraint_targets.append(80.0)

## Initialize all the restraints

In [12]:
for index, atoms in enumerate(static_restraint_atoms):    
    static_DAT_restraint(restraint_mask_list=atoms,
                        num_window_list=windows, 
                        ref_structure=hg,
                        force_constant=static_restraint_angle_fc if len(atoms) > 2 else static_restraint_distance_fc,
                        amber_index=True)

In [13]:
for index, atoms in enumerate(guest_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_final'] = guest_restraint_angle_fc
    else:
        this.attach['fc_final'] = guest_restraint_distance_fc
    this.attach['target'] = guest_restraint_targets[index]
    this.attach['fraction_list'] = attach_fractions
    
    
    this.pull['target_final'] = guest_restraint_target_final[index]
    this.pull['num_windows'] = windows[1]
    this.initialize()
    
    print(amber_restraint_line(this, 'pull', 10))

&rst iat= 1,143,            r1=    0.00000, r2=   10.00000, r3=   10.00000, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 2,1,143,          r1=    0.00000, r2=  180.00000, r3=  180.00000, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 1,143,130,        r1=    0.00000, r2=  180.00000, r3=  180.00000, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end



In [14]:
for index, atoms in enumerate(conformational_restraint_atoms): 
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    this.mask3 = atoms[2]
    this.mask4 = atoms[3]
    
    this.attach['fraction_list'] = attach_fractions
    this.attach['target'] = conformational_restraint_targets[index]
    this.attach['fc_final'] = conformational_restraint_fc
    this.pull['target_final'] = conformational_restraint_targets[index]
    this.pull['num_windows'] = windows[1]
    
    this.initialize()

In [15]:
for index, atoms in enumerate(guest_wall_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_initial'] = guest_wall_restraint_angle_fc
        this.attach['fc_final'] = guest_wall_restraint_angle_fc
        this.custom_restraint_values['rk2'] = 500.0
        this.custom_restraint_values['rk3'] = 0.0
    else:
        this.attach['fc_initial'] = guest_wall_restraint_distance_fc
        this.attach['fc_final'] = guest_wall_restraint_distance_fc
        this.custom_restraint_values['rk2'] = 50.0
        this.custom_restraint_values['rk3'] = 50.0
        this.custom_restraint_values['r1'] = 0.0
        this.custom_restraint_values['r2'] = 0.0

    this.attach['target'] = guest_wall_restraint_targets[index]
    this.attach['num_windows'] = len(attach_fractions)
    this.initialize()

## Make the window list

In [16]:
window_list = create_window_list(DAT_restraint.instances)
print(window_list)

['a000', 'a001', 'a002', 'a003', 'a004', 'a005', 'a006', 'a007', 'a008', 'a009', 'a010', 'a011', 'a012', 'a013', 'p000', 'p001', 'p002', 'p003', 'p004', 'p005', 'p006', 'p007', 'p008', 'p009', 'p010', 'p011', 'p012', 'p013', 'p014', 'p015', 'p016', 'p017', 'p018', 'p019', 'p020', 'p021', 'p022', 'p023', 'p024', 'p025', 'p026', 'p027', 'p028', 'p029', 'p030', 'p031', 'p032', 'p033', 'p034', 'p035', 'p036', 'p037', 'p038', 'p039', 'p040', 'p041', 'p042', 'p043', 'p044', 'p045']


In [17]:
make_window_dirs(window_list)

In [21]:
for window in window_list:
    with open('systems/a-bam-p/confirm-original/' + window + '/disang.rest', 'w') as file:
        for restraint in DAT_restraint.instances:
            if window[0] == 'a':
                phase = 'attach'
            if window[0] == 'p':
                phase = 'pull'
            if window[0] == 'r':
                phase = 'release'
            # print(amber_restraint_line(restraint, phase, int(window[1:])))
            string = amber_restraint_line(restraint, phase, int(window[1:]))
            print(string)
            if string is not None:
                file.write(string)

&rst iat= 1,13,             r1=    0.00000, r2=    7.49023, r3=    7.49023, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 2,1,13,           r1=    0.00000, r2=  131.36554, r3=  131.36554, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 3,2,1,13,         r1=  -64.88459, r2=  115.11541, r3=  115.11541, r4=  295.11541, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 1,13,46,          r1=    0.00000, r2=   55.11100, r3=   55.11100, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 2,1,13,46,        r1=  -15.40895, r2=  164.59105, r3=  164.59105, r4=  344.59105, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 1,13,46,104,      r1=  -70.84632, r2=  109.15368, r3=  109.15368, r4=  289.15368, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 1,143,            r1=    0.00000, r2=    6.00000, r3=    6.00000, r4=  999.00000, rk2=    0.00000, rk3=    0.00000,  &end

&rst iat= 2,1,143,          r1=    0.00000, r2=  180.00000, r3=  180.

None
None
None
None
None
None
None
None
None
None
&rst iat= 1,13,             r1=    0.00000, r2=    7.49023, r3=    7.49023, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 2,1,13,           r1=    0.00000, r2=  131.36554, r3=  131.36554, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 3,2,1,13,         r1=  -64.88459, r2=  115.11541, r3=  115.11541, r4=  295.11541, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 1,13,46,          r1=    0.00000, r2=   55.11100, r3=   55.11100, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 2,1,13,46,        r1=  -15.40895, r2=  164.59105, r3=  164.59105, r4=  344.59105, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 1,13,46,104,      r1=  -70.84632, r2=  109.15368, r3=  109.15368, r4=  289.15368, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 1,143,            r1=    0.00000, r2=   18.00000, r3=   18.00000, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 2,1,143, 

In [19]:
def compare_disang(reference, target):
 
    with open(reference, 'r') as ref:
        next(ref)
        reference_lines = ref.readlines()
    with open(target, 'r') as tar:
        target_lines = tar.readlines()
               
    for x, y in zip(sorted(reference_lines), sorted(target_lines)):
    #for x, y in zip(reference_lines, target_lines):
        
        x = x.strip().split()
        y = y.strip().split()
        
        if x[2][-1] == ',':
            ref_atoms = x[2][:-1]
        else:
            ref_atoms = x[2]
        ref_r1 = x[4][:-1]
        ref_r2 = x[6][:-1]
        ref_r3 = x[8][:-1]
        ref_r4 = x[10][:-1]
        ref_rk2 = x[12][:-1]
        ref_rk3 = x[14][:-1]
        
        tar_atoms = y[2][:-1]
        tar_r1 = y[4][:-1]
        tar_r2 = y[6][:-1]
        tar_r3 = y[8][:-1]
        tar_r4 = y[10][:-1]
        tar_rk2 = y[12][:-1]
        tar_rk3 = y[14][:-1]
        
        # print(ref_atoms + ' ?=? ' + tar_atoms)
        # print(ref_r1 + ' ?=? ' + tar_r1)

        # print('{:20s}{:20s}'.format(ref_atoms, tar_atoms))
        
        try:
            assert(ref_atoms == tar_atoms)
        except:
            print('Index mismatch.')
            print(ref_atoms, tar_atoms)

        try:
            np.testing.assert_almost_equal(float(ref_r1), float(tar_r1), decimal=2)
        except:
            print('r1 mismatch.')
            print(ref_atoms, ref_r1, tar_atoms, tar_r1)
        try:
            np.testing.assert_almost_equal(float(ref_r2), float(tar_r2), decimal=2)
        except:
            print('r2 mismatch.')
            print(ref_atoms, ref_r2, tar_atoms, tar_r2)
        try:
            np.testing.assert_almost_equal(float(ref_r3), float(tar_r3), decimal=2)
        except:
            print('r3 mismatch.')
            print(ref_atoms, ref_r3, tar_atoms, tar_r3)
        try:
            np.testing.assert_almost_equal(float(ref_r4), float(tar_r4), decimal=2)
        except:
            print('r4 mismatch.')
            print(ref_atoms, ref_r4, tar_atoms, tar_r4)
        try:
            np.testing.assert_almost_equal(float(ref_rk2), float(tar_rk2), decimal=2)
        except:
            print('rk2 mismatch.')
            print(ref_atoms, ref_rk2, tar_atoms, tar_rk2)
        try:
            np.testing.assert_almost_equal(float(ref_rk3), float(tar_rk3), decimal=2)
        except:
            print('rk3 mismatch.')
            print(ref_atoms, ref_rk3, tar_atoms, tar_rk3)

In [20]:
compare_disang('/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a00/original/disang.rest',
              '/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/windows/a000/disang.rest')

FileNotFoundError: [Errno 2] No such file or directory: '/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a00/original/disang.rest'

In [20]:
reference = '/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a*/original/disang.rest'
import glob
references = sorted(glob.glob(reference))
target = '/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/windows/a*/disang.rest'
targets = sorted(glob.glob(target))

for ref, tar in zip(references, targets):
    print(ref)
    compare_disang(ref, tar)

/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a00/original/disang.rest
/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a01/original/disang.rest
/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a02/original/disang.rest
/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a03/original/disang.rest
/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a04/original/disang.rest
/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a05/original/disang.rest
/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a06/original/disang.rest
/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a07/original/disang.rest
/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/a08/orig

In [21]:
reference = '/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/u*/original/disang.rest'
import glob
references = sorted(glob.glob(reference))
target = '/home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/windows/p*/disang.rest'
targets = sorted(glob.glob(target))

for ref, tar in zip(references, targets):
    print(ref.split('/')[-3], tar.split('/')[-2])


u00 p000
u01 p001
u02 p002
u03 p003
u04 p004
u05 p005
u06 p006
u07 p007
u08 p008
u09 p009
u10 p010
u11 p011
u12 p012
u13 p013
u14 p014
u15 p015
u16 p016
u17 p017
u18 p018
u19 p019
u20 p020
u21 p021
u22 p022
u23 p023
u24 p024
u25 p025
u26 p026
u27 p027
u28 p028
u29 p029
u30 p030
u31 p031
u32 p032
u33 p033
u34 p034
u35 p035
u36 p036
u37 p037
u38 p038
u39 p039
u40 p040
u41 p041
u42 p042
u43 p043
u44 p044
u45 p045


In [22]:
for ref, tar in zip(references, targets):
    print(ref.split('/')[-3], tar.split('/')[-2])
    compare_disang(ref, tar)

u00 p000
u01 p001
u02 p002
u03 p003
u04 p004
u05 p005
u06 p006
u07 p007
u08 p008
u09 p009
u10 p010
u11 p011
u12 p012
u13 p013
u14 p014
u15 p015
u16 p016
u17 p017
u18 p018
u19 p019
u20 p020
u21 p021
u22 p022
u23 p023
u24 p024
u25 p025
u26 p026
u27 p027
u28 p028
u29 p029
u30 p030
u31 p031
u32 p032
u33 p033
u34 p034
u35 p035
u36 p036
u37 p037
u38 p038
u39 p039
u40 p040
u41 p041
u42 p042
u43 p043
u44 p044
u45 p045


Next, let's (a) use these parameters to recapitulate the GAFF results and (b) setup the first full SMRINOFF APR calculation.

# Test analysis

In [23]:
simulation_data = {
    'attach'  : [],
    'pull'    : [],
    'release' : []
}

for window in window_list:
    phase_prefix = window[0]
    
    if phase_prefix == 'a':
        phase = 'attach'
    if phase_prefix == 'p':
        phase = 'pull'
    if phase_prefix == 'r':
        phase = 'release'
        
    window_index = int(window[1:])
        
    simulation_data[phase].append([])
    #simulation_data[phase][window_index] = []
        
    # for restraint_index, restraint in enumerate(DAT_restraint.instances[6]):
    
    for restraint_index, restraint in enumerate(DAT_restraint.instances[:-13]):

        simulation_data[phase][window_index].append([]) 
        # simulation_data[phase][window_index][restraint_index] = []

        structure = pt.load(os.path.join('systems/a-bam-p/confirm-original/', window, 'full.crds'), 
                           os.path.join('systems/a-bam-p/confirm-original/', window, 'full.hmr.topo'))

        stripped = structure.strip(':WAT,:Na+,:Cl-')

        traj = pt.iterload(os.path.join('systems/a-bam-p/confirm-original', window, 'traj.nc'),
                          top=stripped.topology)

        if restraint.mask1 and restraint.mask2 and \
        not restraint.mask3 and not restraint.mask4:
            data = pt.distance(traj, ' '.join([restraint.mask1, restraint.mask2]))
        elif restraint.mask1 and restraint.mask2 and \
        restraint.mask3 and not restraint.mask4:
            data = pt.angle(traj, ' '.join([restraint.mask1, restraint.mask2, restraint.mask3]))
        elif restraint.mask1 and restraint.mask2 and \
        restraint.mask3 and restraint.mask4:
            data = pt.dihedral(traj, ' '.join([restraint.mask1, restraint.mask2, restraint.mask3, restraint.mask4]))

        simulation_data[phase][window_index][restraint_index] = data

In [24]:
simulation_data

{'attach': [[array([ 7.61686122,  7.93255718,  7.40951648, ...,  7.34328699,
           7.43030807,  7.40921703]),
   array([ 128.18361046,  133.98495524,  128.1042591 , ...,  133.97279403,
           134.80317092,  134.3749529 ]),
   array([ 115.0264328 ,  113.58483303,  113.60870957, ...,  115.91210259,
           114.62024571,  111.41529616]),
   array([ 53.60475312,  53.10847813,  50.90089557, ...,  50.50763025,
           52.53172468,  50.14637498]),
   array([ 170.34681096,  165.52247744,  165.95032522, ...,  155.58514695,
           162.05264565,  166.03302768]),
   array([ 108.27541947,  103.68453555,  108.22996182, ...,  115.26423334,
           117.09334338,  115.32741178]),
   array([ 5.60203595,  5.29801344,  4.88877366, ...,  6.44187204,
           5.8067868 ,  5.52575785]),
   array([ 169.51721037,  171.32129103,  168.55737585, ...,  167.17462803,
           169.12981104,  169.04559672]),
   array([ 167.28046861,  161.4428815 ,  152.75060999, ...,  163.5999436 ,
         

In [25]:
from paprika.analysis import fe_calc
analysis = fe_calc()
analysis.restraint_list = DAT_restraint.instances[:-13]
analysis.simulation_values = simulation_data
analysis.compute_free_energy()

!!!!!!!!!!!!!!
[False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
!!!!!!!!!!!!!!
[False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
!!!!!!!!!!!!!!
[False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
!!!!!!!!!!!!!!
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [26]:
analysis.results

{'attach': {'mbar-block': {'convergence': array([ 0.01227673,  0.01227673,  0.00561205, ...,  0.03534279,
           0.03534279,  0.02970423]),
   'fe': 10.475734326371544,
   'fe_matrix': array([[  0.        ,   0.18993472,   0.35655006, ...,   7.5475985 ,
             9.22146913,  10.47573433],
          [ -0.18993472,   0.        ,   0.16661535, ...,   7.35766378,
             9.03153442,  10.28579961],
          [ -0.35655006,  -0.16661535,   0.        , ...,   7.19104844,
             8.86491907,  10.11918426],
          ..., 
          [ -7.5475985 ,  -7.35766378,  -7.19104844, ...,   0.        ,
             1.67387063,   2.92813583],
          [ -9.22146913,  -9.03153442,  -8.86491907, ...,  -1.67387063,
             0.        ,   1.25426519],
          [-10.47573433, -10.28579961, -10.11918426, ...,  -2.92813583,
            -1.25426519,   0.        ]]),
   'ordered_convergence': array([ 0.01227673,  0.01227673,  0.00561205, ...,  0.03534279,
           0.03534279,  0.02970423